In [2]:
%reload_ext autoreload
%autoreload 2

import logging
import torch 
import torch.nn as nn
import os
dir = "/home/cmk2000/Documents/Years 3/Python codes/Codes/LLMs/Conditional-Diffusion-Model-for-IVP"
os.chdir(dir)
from models.model import FineTuningDiffusionModel


from guided_diffusion import dist_util
from guided_diffusion.script_util import (
    NUM_CLASSES,
    model_and_diffusion_defaults,
    create_model_and_diffusion,
    args_to_dict,
)

from utils import utils_model

import datetime

from guided_diffusion import dist_util
from guided_diffusion.script_util import (
    NUM_CLASSES,
    model_and_diffusion_defaults,
    create_model_and_diffusion,
    args_to_dict,
)


from guided_diffusion.unet import *
    
from utils_lora.lora_parametrisation import LoRa_model,make_copy_model
import warnings

warnings.filterwarnings("ignore")

/home/cmk2000/.conda/envs/horus_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Matplotlib is building the font cache; this may take a moment.


In [3]:
def total_parameters(model):
    n_frozen = 0
    n_lora = 0
    for name, param in model.named_parameters():
        if "lora_" not in name:
            n_frozen += param.numel()
        else:
            n_lora += param.numel()
    return n_frozen, n_lora

In [4]:
# ----------------------------------------
# Preparation
# ----------------------------------------

date = datetime.datetime.now().strftime("%d%B%Y")

device                  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

noise_level_img         = 12.75/255.0           # set AWGN noise level for LR image, default: 0
noise_level_model       = noise_level_img       # set noise level of model, default: 0
model_name              = 'diffusion_ffhq_10m' #'diffusion_ffhq_10m'  # diffusion_ffhq_10m, 256x256_diffusion_uncond; set diffusino model
testset_name            = 'demo_test'            # set testing set,  'imagenet_val' | 'ffhq_val'
num_train_timesteps     = 1000
iter_num                = 1000              # set number of iterations
iter_num_U              = 1                 # set number of inner iterations, default: 1
skip                    = num_train_timesteps//iter_num     # skip interval


log_process             = False
ddim_sample             = False             # sampling method
model_output_type       = 'pred_xstart'     # model output type: pred_x_prev; pred_xstart; epsilon; score
generate_mode           = 'DiffPIR'         # DiffPIR; DPS; vanilla
skip_type               = 'quad'            # uniform, quad
eta                     = 0.0               # eta for ddim sampling
zeta                    = 0.1  
guidance_scale          = 1.0   

calc_LPIPS              = True
use_DIY_kernel          = True
blur_mode               = 'Gaussian'          # Gaussian; motion      
kernel_size             = 9
kernel_std              = 3.0 if blur_mode == 'Gaussian' else 0.5

if blur_mode=='Gaussian':
    kernel_true_param   = .4
else:
    raise Exception("Hol' up, only Gaussian bois allowed here.")



sf                      = 1
task_current            = 'deblur'          
n_channels              = 3                 # fixed
cwd                     = ''  
model_zoo               = os.path.join(cwd, '/home/cmk2000/Documents/Years 2/Python codes and results/Python codes/Codes/pretrained_models/model_zoo')    # fixed
testsets                = os.path.join(cwd, 'testsets')     # fixed
model_path              = os.path.join(model_zoo, model_name+'.pt')

torch.cuda.empty_cache()

# noise schedule 
beta_start              = 0.1 / 1000
beta_end                = 20 / 1000
betas                   = np.linspace(beta_start, beta_end, num_train_timesteps, dtype=np.float32)
betas                   = torch.from_numpy(betas).to(device)
alphas                  = 1.0 - betas
alphas_cumprod          = np.cumprod(alphas.cpu(), axis=0)
sqrt_alphas_cumprod     = torch.sqrt(alphas_cumprod)
sqrt_1m_alphas_cumprod  = torch.sqrt(1. - alphas_cumprod)
reduced_alpha_cumprod   = torch.div(sqrt_1m_alphas_cumprod, sqrt_alphas_cumprod)        # equivalent noise sigma on image

noise_model_t           = utils_model.find_nearest(reduced_alpha_cumprod, 2 * noise_level_model)
noise_model_t           = 0

noise_inti_img          = 50 / 255
t_start                 = utils_model.find_nearest(reduced_alpha_cumprod, 2 * noise_inti_img) # start timestep of the diffusion process
t_start                 = num_train_timesteps - 1              

# ----------------------------------------
# L_path, E_path, H_path
# ----------------------------------------

L_path = os.path.join(testsets, testset_name) # L_path, for Low-quality images


# ----------------------------------------
# load model
# ----------------------------------------
print(model_name)
model_config = dict(
        model_path=model_path,
        num_channels=128,
        num_res_blocks=1,
        attention_resolutions="16",
    ) if model_name == 'diffusion_ffhq_10m' \
    else dict(
        model_path=model_path,
        num_channels=256,
        num_res_blocks=2,
        attention_resolutions="8,16,32",
    )
    
args = utils_model.create_argparser(model_config).parse_args([])
model, diffusion = create_model_and_diffusion(
    **args_to_dict(args, model_and_diffusion_defaults().keys()))
model.load_state_dict(
    dist_util.load_state_dict(args.model_path, map_location="cpu")
)
model.eval()
for _k, v in model.named_parameters():
    v.requires_grad = False
model = model.to(device)


model_copy = make_copy_model(model)
lora_model = LoRa_model(model_copy, input_block_layer_lora = [9, 9], middle_block_layer_lora = [1], output_block_layer_lora = [2, 3])

total_parameters(model), total_parameters(lora_model)



diffusion_ffhq_10m
Number of Layers frozen: 362
Total trainable parameters of the model: 93563910 (non-LoRa) vs 24576 (LoRa) Ratio: 0.03% of the original model


((93563910, 0), (93563910, 24576))

In [5]:
model

UNetModel(
  (time_embed): Sequential(
    (0): Linear(in_features=128, out_features=512, bias=True)
    (1): SiLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
  )
  (input_blocks): ModuleList(
    (0): TimestepEmbedSequential(
      (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1): TimestepEmbedSequential(
      (0): ResBlock(
        (in_layers): Sequential(
          (0): GroupNorm32(32, 128, eps=1e-05, affine=True)
          (1): SiLU()
          (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (h_upd): Identity()
        (x_upd): Identity()
        (emb_layers): Sequential(
          (0): SiLU()
          (1): Linear(in_features=512, out_features=256, bias=True)
        )
        (out_layers): Sequential(
          (0): GroupNorm32(32, 128, eps=1e-05, affine=True)
          (1): SiLU()
          (2): Dropout(p=0.1, inplace=False)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stri

In [3]:


total_parameters(model)

(93563910, 0)

In [48]:
image_size = 256
in_channels = 3
model_channels = 128
out_channels = 3

model_L = FineTuningDiffusionModel(image_size, in_channels, model_channels, out_channels, pt_model = lora_model)


#total_parameters(model)
model_L
total_parameters(model_L)[0] - total_parameters(model)[0], total_parameters(model_L)

(34119683, (127683593, 24576))

In [6]:
import cv2
pht = "/home/cmk2000/Documents/Years 3/Python codes/Codes/image/69037.png"
x = torch.tensor(cv2.imread(pht,0))/255
x = x.unsqueeze(0).unsqueeze(0).repeat(2,3,1,1)
x.shape

torch.Size([2, 3, 256, 256])

In [7]:



model_L(x, timesteps=torch.tensor([4]))

Started ...


tensor([[[[-4.4464e-01, -8.0025e-02, -5.8108e-02,  ...,  5.4666e-02,
            6.3529e-02,  1.1771e-01],
          [ 1.0024e-01, -3.1844e-02, -4.6185e-01,  ...,  6.8132e-01,
           -1.0140e+00,  6.5924e-01],
          [-1.2030e-01,  1.1550e-02, -2.7586e-01,  ...,  2.7033e-01,
           -8.7674e-02,  5.1268e-01],
          ...,
          [-1.6690e-01, -1.8327e-01, -2.4938e-01,  ..., -7.5878e-02,
           -9.3964e-02, -1.1954e-01],
          [-5.7894e-01,  7.0161e-02,  6.6814e-02,  ..., -2.1156e-01,
           -3.8093e-01,  2.7093e-02],
          [ 3.1853e-01,  3.4688e-01,  3.6710e-01,  ...,  3.8196e-01,
            5.0094e-01, -8.0849e-02]],

         [[ 3.3068e-01,  4.9126e-02,  5.3883e-02,  ..., -1.7130e-01,
            1.9943e-01,  8.8986e-01],
          [ 3.5806e-01,  2.1796e-01,  4.4262e-01,  ...,  9.1358e-01,
            2.7440e-01,  4.1025e-01],
          [ 1.3305e-01,  3.6046e-01,  9.5942e-02,  ..., -8.2276e-02,
            3.7286e-01,  4.4076e-01],
          ...,
     

In [ ]:
model.out(torch.randn(1,128,256,256))

In [ ]:
pht = "/home/cmk2000/Documents/Years 3/Python codes/Codes/image/69037.png"

In [12]:
total_parameters(model_L)

(168612620, 24576)

In [11]:
import yaml

# Load the YAML file
with open('/home/cmk2000/Documents/Years 3/Python codes/Codes/LLMs/Fine tuning  diffusion models/configs/training_fine_tuning_lora_model.yaml', 'r') as file:
    configs = yaml.safe_load(file)



TypeError: print() takes at most 4 keyword arguments (15 given)

1 3 4 5 6 7


In [17]:
import blobfile as bf
from mpi4py import MPI
import torch as th
import torch.distributed as dist
import os

In [18]:
dist.is_initialized()

False

In [19]:
os.environ["CUDA_VISIBLE_DEVICES"]

KeyError: 'CUDA_VISIBLE_DEVICES'

In [1]:

import torch.distributed as dist

dist.get_world_size()

ValueError: Default process group has not been initialized, please make sure to call init_process_group.

In [2]:
import torch

In [3]:
a = torch.randn(12,12)

In [4]:
a.dtype

torch.float32

In [24]:
import random
random.randint(0, 100)

82

In [50]:
import torchvision.transforms as transforms #type : ignore
tra = transforms.Compose([transforms.ToPILImage(), transforms.Resize((256,256)), transforms.ToTensor()])

In [7]:
from distutils.util import strtobool

In [22]:
import torchvision as transforms #type : ignore
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import glob
import os

In [20]:
pth = "/home/cmk2000/Documents/Years 3/Python codes/Codes/image/69037.png"
pt = "/home/cmk2000/Documents/Years 3/Python codes/Codes/image"

x = np.array(Image.open(pth))
x.shape

(256, 256, 3)

In [37]:
l = glob.iglob(os.path.join(pt, '*.png'))
l.values

AttributeError: 'generator' object has no attribute 'values'

In [35]:
l[0].rsplit("/", 1)[-1]

TypeError: 'generator' object is not subscriptable

In [25]:
from accelerate import Accelerator

/home/cmk2000/.conda/envs/horus_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
import torch
from PIL import Image
from torchvision import transforms
import numpy as np

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]),
])

In [28]:
image = Image.open("/home/cmk2000/Documents/Years 3/Python codes/Codes/image/69037.png")
print(np.array(image).shape, image.size)
transform(image).shape

(256, 256, 3) (256, 256)


torch.Size([3, 256, 256])

torch.Size([2, 3, 5])

In [12]:
(256 // 16)

16

In [15]:
import torch.nn as nn
class test(nn.Module):
    def __init__(self, x):
        self.x = x
        self.modules = 12
    def __call__(self, y):
        return self.x + y + self.modules

t = test(3)
t(4)

19